# ELEC 474 Lab2: K-Means Clustering

Matthieu Roux - 20013052

For Lab 2 we will be implementing a k-means clustering algorithm. We will perform color quantization on an image using k-means with random center(s) initialization

## Imports

In [20]:
import cv2
import numpy as np
import copy
import math

img_path = "mini_baboon.jpg"
img = cv2.imread(img_path)

img_height = img.shape[0]
img_width = img.shape[1]
supbixel_range = img.shape[2]

 You should create a track bar to select and experiment with different values of 𝒌.
Your randomly selected cluster centers, { ci}, i = 1..k, should include color channel intensities, as
they will be used to calculate your means.

In [11]:
# code for step 1 here

# generate a k, the number of clusters (colors) we will have
max_k_value = 10 # we will set a max value for k so that we dont generate an out of control amount of clusters
k = np.random.randint(low=1, high=max_k_value)
print("We will divide the image in", k, "colors!") 

# then randomly select points 



We will divide the image in 9 colors!


## Step 2
Partition your image into Voronoi Regions based on the color values of each pixel. 

The Voronoi Region around each cluster center ci includes all pixels that are closest to that cluster center,
according to some distance metric. In this case, as we are only interested in the color values of
the pixels, you should use a distance metric based solely on the color channels to determine
which pixels belong to which cluster.


In [12]:
# code for step 2

"""
euclidean_distance(element_1, element_2)
    Returns the euclidean distance between point 1 and 2 point 1 and 2 are 2 arrays of length n. n represents the dimension of points 1 and 2. 
"""
def euclidean_distance(element_1, element_2):
    return math.sqrt(sum([(a - b) ** 2 for a, b in zip(element_1, element_2)]))

def compute_voronoids(cluster_centers):
    global img
    pixel_to_region = {}
    for row in img:
        for pixel in row:
            euclidean_distances = [euclidean_distance(cluster_center, pixel) for cluster_center in cluster_centers]
        
            # add the pixel to its closest region
            pixel_to_region[pixel.tobytes()] = euclidean_distances.index(min(euclidean_distances)) - 1
    return pixel_to_region


## Step 3

Find the mean color value mi for each Voronoi Region.

a) If for each region, the mean values are very close to the cluster centers (i.e. mi ci for all i =
1..k ), then the algorithm has converged. Recolor all pixels in each region to their mean color
values, and STOP.

b) Else, if any of the mean values differ significantly from their cluster centers (i.e. mi  ci for any
1..k), then update the cluster centers to equal the mean values (i.e. set ci = mi) and repeat
Steps 2 and 3.

In [13]:
# code for step 3

def compute_region_mean(regions):
    global supbixel_range, k
    region_sizes = region_totals = np.zeros((k,supbixel_range), np.uint8)
    for row in img:
        for pixel in row:
            region_index = regions[pixel.tobytes()]
            region_totals[region_index] = np.add(region_totals[region_index], pixel)  
            region_sizes[region_index] = region_sizes[region_index] + 1
    return [total/size for total, size in zip(region_totals, region_sizes)]

def is_within_threshold(threshold, cluster_centers, region_means):
    return False


def compute_clusters(k, threshold = 30, max_loops = 3):
    # compute cluster centers
    cluster_indexes = [( np.random.randint(low=0, high=img_height), np.random.randint(low=0, high=img_width)) for i in range(k)]
    cluster_centers = [img[index] for index in cluster_indexes]

    for loop in range(max_loops):
        # get the voronoid regions
        pixel_to_region_dict = compute_voronoids(cluster_centers)

        # get the region means
        region_means = compute_region_mean(pixel_to_region_dict)

        if is_within_threshold(threshold, cluster_centers, region_means):
            print("Segmentation took", loop, "runs")
            return cluster_centers, pixel_to_region_dict
        # if we are not within the threshold update the cluster_center values with means
        cluster_centers = region_means
    return cluster_centers, pixel_to_region_dict
        




In [23]:
def update_img(k, img):
    cluster_centers, pixel_to_region_dict = compute_clusters(k)
    for y in range(img_height):
        for x in range(img_width):
            img[y][x] = cluster_centers[pixel_to_region_dict[img[y][x].tobytes()]]
    return img

In [24]:

# on_trackbar is called when the trackbar is changed, it updates the threshold
def on_trackbar(val):
    global k
    k = val
    reset_all()

# reset all is used to reset the image
def reset_all():
    global img
    img = cv2.imread(img_path)

# Create the trackbar that will allow users to edit k
# cv2.createTrackbar("k", window_name, k , max_k_value, on_trackbar)

# update image
cluster_centers, pixel_to_region_dict = compute_clusters(k)
img = update_img(k, img)

# Rendering
window_name = img_path
cv2.namedWindow(window_name)

while(True):
    
    # Wait a little bit for the image to re-draw
    key = cv2.waitKey(1)
    cv2.imshow(window_name, img)
    
    # If an x is pressed, the window will close
    if key == ord("x"):
        break
        
    # If r is pressed, the image adn regions will reset.
    if key == ord("r"):
        img = cv2.imread(img_path)
        reset_all()


In [25]:
# Run this if the open cv window starts responding / behaving
cv2.destroyAllWindows()